# **EVALUACION 1 MINERIA DE DATOS 2024 (BIY7121_004V) - STARCRAFT II**

In [3]:
# Importamos librerias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import KNNImputer

## **1. BUSSINESS UNDERSTANDING**

Se nos ha entregado un dataset correspondiente a información de partidas multijugador de Starcraft II, juego RTS (Real Time Strategy) lanzado en 2010 como una secuela del juego lanzado originalmente en 1998. 

In [4]:
# Cargamos el dataset
sc2DF = pd.read_csv("starcraft_duoc.csv", delimiter=';')

## **2. DATA UNDERSTANDING**

In [5]:
sc2DF.shape

(3395, 20)

In [ ]:
# Mostramos los primeros 5 registros
sc2DF.head()

In [ ]:
# Los ultimos 5 registros
sc2DF.tail()

In [ ]:
# Una vista a los tipos de datos de cada una de las columnas
sc2DF.dtypes

De esta revisión es posible observar lo siguiente:
* Se da cuenta de que no existen variables categóricas en nuestro data set, por lo que no es necesario hacer un proceso de encoding de dichas variables.
* Sin embargo, podemos observar que muchas columnas de nuestro dataset presentan problemas de formato. Dicho de otro modo, es posible notar que hay números, sean enteros (int) o decimales (float), que están bajo formato de texto (string). Se hace necesario, por tanto, hacer un tratamiento para darles el formato correcto.

In [ ]:
def isset(x):
    try:
        x
        return True
    except:
        pass

for column in sc2DF.columns:
    for row in range(len(sc2DF)):
        try:
            if isinstance(sc2DF[column][row], str) and sc2DF[column][row][-4] == '.':
                sc2DF[column][row] = sc2DF[column][row].replace('.', '')
        except:
            print(sc2DF[column][row])
    sc2DF[column] = sc2DF[column].astype(float)


### 2.1. **Limpieza y transformación**

#### 2.1.1. **Tratamiento de valores nulos**

In [ ]:
sc2DF.isna().sum()

In [ ]:
# Vamos a imputar valores usando KNN
# Crear una instancia de KNNImputer
imputer = KNNImputer(n_neighbors=3, weights='distance') #uniform

# Imputar los valores faltantes
imputed_data = imputer.fit_transform(sc2DF)
sc2DFImputed = pd.DataFrame(imputed_data)

In [ ]:
sc2DF.isna().sum()

#### 2.1.2 **Tratamiento de outliers**

### 2.3. **Análisis y procesamiento**

In [ ]:
array = ['1.000.000', 566.32, 56.345, '897.43']

for i in range(len(array)):
    if isinstance(array[i], str):
        # Eliminar los puntos como separadores de miles
        array[i] = float(array[i].replace('.', ''))
    else:
        array[i] = float(array[i])

print(array)